 #### Nestable Lock Routines

 The following example demonstrates how a nestable lock can be used to synchronize  updates both to a whole structure and to one of its members.

In [ ]:

/*
* name: nestable_lock.1c
* type: C
*/

#include <omp.h>

typedef struct {
   int a,b;
   omp_nest_lock_t lck;
} pair;

int work1();
int work2();
int work3();

void incr_a(pair *p, int a)
{

  /* Called only from incr_pair, no need to lock. */
  p->a += a;

}

void incr_b(pair *p, int b)
{

  /* Called both from incr_pair and elsewhere, */
  /* so need a nestable lock. */

  omp_set_nest_lock(&p->lck);
  p->b += b;
  omp_unset_nest_lock(&p->lck);

}

void incr_pair(pair *p, int a, int b)
{

  omp_set_nest_lock(&p->lck);
  incr_a(p, a);
  incr_b(p, b);
  omp_unset_nest_lock(&p->lck);

}

void nestlock(pair *p)
{

  #pragma omp parallel sections
  {
    #pragma omp section
      incr_pair(p, work1(), work2());
    #pragma omp section
      incr_b(p, work3());
  }

}



In [ ]:

! name: nestable_lock.1f
! type: F-fixed
      MODULE DATA
        USE OMP_LIB, ONLY: OMP_NEST_LOCK_KIND
        TYPE LOCKED_PAIR
          INTEGER A
          INTEGER B
          INTEGER (OMP_NEST_LOCK_KIND) LCK
       END TYPE
      END MODULE DATA

      SUBROUTINE INCR_A(P, A)
        ! called only from INCR_PAIR, no need to lock
        USE DATA
        TYPE(LOCKED_PAIR) :: P
        INTEGER A
        P%A = P%A + A
      END SUBROUTINE INCR_A

      SUBROUTINE INCR_B(P, B)
        ! called from both INCR_PAIR and elsewhere,
        ! so we need a nestable lock
        USE OMP_LIB       ! or INCLUDE "omp_lib.h"
        USE DATA
        TYPE(LOCKED_PAIR) :: P
        INTEGER B
        CALL OMP_SET_NEST_LOCK(P%LCK)
        P%B = P%B + B
        CALL OMP_UNSET_NEST_LOCK(P%LCK)
      END SUBROUTINE INCR_B

      SUBROUTINE INCR_PAIR(P, A, B)
        USE OMP_LIB        ! or INCLUDE "omp_lib.h"
        USE DATA
        TYPE(LOCKED_PAIR) :: P
        INTEGER A
        INTEGER B

        CALL OMP_SET_NEST_LOCK(P%LCK)
        CALL INCR_A(P, A)
        CALL INCR_B(P, B)
        CALL OMP_UNSET_NEST_LOCK(P%LCK)
      END SUBROUTINE INCR_PAIR

      SUBROUTINE NESTLOCK(P)
        USE OMP_LIB        ! or INCLUDE "omp_lib.h"
        USE DATA
        TYPE(LOCKED_PAIR) :: P
        INTEGER WORK1, WORK2, WORK3
        EXTERNAL WORK1, WORK2, WORK3

!$OMP   PARALLEL SECTIONS

!$OMP   SECTION
          CALL INCR_PAIR(P, WORK1(), WORK2())
!$OMP   SECTION
          CALL INCR_B(P, WORK3())
!$OMP   END PARALLEL SECTIONS

      END SUBROUTINE NESTLOCK

